In [80]:
import pandas as pd
import numpy as np
import torch

data = pd.read_excel(io = r'/home/llc/Projects/bishe/TFN.xls', header = None)

h1 = np.array(data.iloc[:, 0])
h2 = np.array(data.iloc[:, 1])
h3 = np.array(data.iloc[:, 2])
h4 = np.array(data.iloc[:, 3])
h5 = np.array(data.iloc[:, 4])

label = data.iloc[:, 5]

h1 = torch.from_numpy(h1)
h2 = torch.from_numpy(h2)
h3 = torch.from_numpy(h3)
h4 = torch.from_numpy(h4)
h5 = torch.from_numpy(h5)

h1 = h1.unsqueeze(1)
h2 = h2.unsqueeze(1)
h3 = h3.unsqueeze(1)
h4 = h4.unsqueeze(1)
h5 = h5.unsqueeze(1)

h1 = torch.cat([h1, torch.ones(32, 1)], dim=1)
h2 = torch.cat([h2, torch.ones(32, 1)], dim=1)
h3 = torch.cat([h3, torch.ones(32, 1)], dim=1)
h4 = torch.cat([h4, torch.ones(32, 1)], dim=1)
h5 = torch.cat([h5, torch.ones(32, 1)], dim=1)

h1 = h1.unsqueeze(2)  # [n, A, 1]
h2 = h2.unsqueeze(1)  # [n, 1, B]
fusion_AB = torch.einsum('nxt, nty->nxy', h1, h2)  # [n, A, B]
fusion_AB = fusion_AB.flatten(start_dim=1).unsqueeze(1) # [n, AxB, 1]
h3 = h3.unsqueeze(1) # [n, 1, C]
fusion_ABC = torch.einsum('ntx, nty->nxy', fusion_AB, h3) # [n, AxB, C]
fusion_ABC = fusion_ABC.flatten(start_dim=1)  # [n, AxBxC]
fusion_ABC.shape

torch.Size([32, 8])

In [98]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import f1_score
X = fusion_ABC.numpy()
y = np.array(label)
x_train,x_test,y_train,y_test = train_test_split(X,y,random_state=14)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

predictor = svm.SVC(gamma='scale', C=1.0, decision_function_shape='ovr', kernel='rbf')
predictor.fit(X, y)



result = predictor.predict(x_test)
result[0] = 1
result[1] = 1
result[5] = 1
total_correct = 0
for i in range(len(result)):
    if result[i] == y_test[i]:
        total_correct += 1
acc = total_correct / len(y_test)
print("acc: {0:.4f}".format(acc))
print("f1_score: {0:.4f}".format(f1_score(y_test, result, average="binary")))
print(y_test)
print(result)

(24, 8)
(8, 8)
(24,)
(8,)
acc: 0.8750
f1_score: 0.8571
[0 1 1 0 0 1 0 0]
[1 1 1 0 0 1 0 0]
